In [5]:
%matplotlib inline
#%matplotlib notebook
#%matplotlib widget
import matplotlib 
import numpy as np
import pandas as pd
import os, sys, time
#import ipywidgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection    
 
import torch
# use LaTeX, choose nice some looking fonts and tweak some settings
matplotlib.rc('font', family='serif')
matplotlib.rc('font', size=16)
matplotlib.rc('legend', fontsize=16)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('legend', handlelength=1.5)
matplotlib.rc('legend', frameon=False)
matplotlib.rc('xtick.major', pad=7)
matplotlib.rc('xtick.minor', pad=7)
matplotlib.rc('text', usetex=True)
# matplotlib.rc('text.latex', 
#               preamble=[r'\usepackage[T1]{fontenc}',
#                         r'\usepackage{amsmath}',
#                         r'\usepackage{txfonts}',
#                         r'\usepackage{textcomp}'])

matplotlib.rc('figure', figsize=(12, 9))

In [27]:
def torchtest(dev="cpu"):
    if dev not in ["cpu", "cuda:0", "cuda:1"]:
        return None
    dtype = torch.float
    device = torch.device(dev)
    # device = torch.device("cuda:0") # GPU에서 실행하려면 이 주석을 제거하세요.
    t1=time.time()
    # N은 배치 크기이며, D_in은 입력의 차원입니다;
    # H는 은닉층의 차원이며, D_out은 출력 차원입니다.
    N, D_in, H, D_out = 64, 1000, 100, 10

    # 무작위의 입력과 출력 데이터를 생성합니다.
    x = torch.randn(N, D_in, device=device, dtype=dtype)
    y = torch.randn(N, D_out, device=device, dtype=dtype)

    # 무작위로 가중치를 초기화합니다.
    w1 = torch.randn(D_in, H, device=device, dtype=dtype)
    w2 = torch.randn(H, D_out, device=device, dtype=dtype)

    learning_rate = 1e-6
  
    for t in range(500):
        # 순전파 단계: 예측값 y를 계산합니다.
        h = x.mm(w1)
        h_relu = h.clamp(min=0)
        y_pred = h_relu.mm(w2)

        # 손실(loss)을 계산하고 출력합니다.
        loss = (y_pred - y).pow(2).sum().item()
        if t % 100 == 99:
            print(t, loss)

        # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
        grad_y_pred = 2.0 * (y_pred - y)
        grad_w2 = h_relu.t().mm(grad_y_pred)
        grad_h_relu = grad_y_pred.mm(w2.t())
        grad_h = grad_h_relu.clone()
        grad_h[h < 0] = 0
        grad_w1 = x.t().mm(grad_h)

        # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다.
        w1 -= learning_rate * grad_w1
        w2 -= learning_rate * grad_w2
    print(time.time()-t1, "secs for execute by "+dev+" .")

In [28]:
torchtest("cuda:0")

99 nan
199 nan
299 nan
399 nan
499 nan
0.22258901596069336 secs for execute by cuda:0 .


In [29]:
torchtest("cpu")

99 nan
199 nan
299 nan
399 nan
499 nan
7.406389474868774 secs for execute by cpu .


In [31]:
torchtest("cuda:1")

99 nan
199 nan
299 nan
399 nan
499 nan
0.1970672607421875 secs for execute by cuda:1 .


In [32]:
x.shape, w1.shape

(torch.Size([64, 1000]), torch.Size([1000, 100]))

In [33]:
x@w1

tensor([[ 25.2480,  35.7939,  -1.6933,  ..., -25.3238,  20.6352, -14.1234],
        [ 33.2458,  -0.7616,  -4.7463,  ..., -43.0959, -29.4794,   0.3630],
        [ 22.3756, -20.5740,  37.1580,  ..., -23.2816,  -0.6073,   0.2749],
        ...,
        [ 10.8524,  -1.4147,  -9.8913,  ...,  24.1980,   5.9258,  28.1207],
        [ 15.8510, -34.1364,  -0.4786,  ...,  19.6586,  -1.6031,   6.6792],
        [ 22.3780, -53.2425, -30.6140,  ..., -22.0529,   6.9655,  31.2972]])

In [34]:
x.mm(w1)

tensor([[ 25.2480,  35.7939,  -1.6933,  ..., -25.3238,  20.6352, -14.1234],
        [ 33.2458,  -0.7616,  -4.7463,  ..., -43.0959, -29.4794,   0.3630],
        [ 22.3756, -20.5740,  37.1580,  ..., -23.2816,  -0.6073,   0.2749],
        ...,
        [ 10.8524,  -1.4147,  -9.8913,  ...,  24.1980,   5.9258,  28.1207],
        [ 15.8510, -34.1364,  -0.4786,  ...,  19.6586,  -1.6031,   6.6792],
        [ 22.3780, -53.2425, -30.6140,  ..., -22.0529,   6.9655,  31.2972]])